# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [ ]:
# Load in additional functions
library(tidyverse)
library(lubridate)

# Use three digits past the decimal point,
# and don't use scientific notation.
options(digits = 3, scipen = 999)

# Format plots with a white background and dark features.
theme_set(theme_bw())

# Increase the default text size of plots.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
theme_update(text = element_text(size = 20))

# Increase the default plot width and height.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
options(repr.plot.width=12, repr.plot.height=8)

# Read in the data
spam = read_csv('https://jdgrossman.com/assets/spam.csv')

# peek at 10 random rows
sample_n(spam, 10)

## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [1]:
# Your code here!

library(tidyverse)

# Load the data
spam <- read_csv("https://jdgrossman.com/assets/spam.csv")

### 1. How many emails are in the database?
n_emails <- nrow(spam)
n_emails

### 2. What fraction of the emails in the database are spam?
fraction_spam <- mean(spam$spam)
fraction_spam

### 3. Which email has the highest percentage of words matching "money"?
# Find row with highest money fraction
max_money_row <- spam %>%
  arrange(desc(money)) %>%
  slice(1)

max_money_row

# Extract the exact percentage
highest_money_percentage <- max(spam$money)
highest_money_percentage


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Rows: 4601 Columns: 58
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (58): make, address, all, 3d, our, over, remove, internet, order, mail, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 4601

Warning message:
“Unknown or uninitialised column: `spam`.”
Warning message in mean.default(spam$spam):
“argument is not numeric or logical: returning NA”


[1] NA

make,address,all,3d,our,over,remove,internet,order,mail,⋯,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,1,1,2,1


[1] 12.5

## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [7]:
library(tidyverse)

library(tidyverse)

# Load data
spam <- read_csv("https://jdgrossman.com/assets/spam.csv")

# --- Step 1: Identify columns with very few unique values ---
unique_counts <- sapply(spam, function(x) length(unique(x)))

print("Unique value counts per column:")
print(sort(unique_counts))

# --- Step 2: Pick the response column automatically ---
# This selects the FIRST column with <= 2 unique values
target_col <- names(unique_counts[unique_counts <= 2])[1]

cat("Detected spam indicator column:", target_col, "\n")

# --- Step 3: Create binary outcome spam_binary ---
# If it's numeric (0/1), we keep it
# If it's character, convert it to 0/1
spam <- spam %>%
  mutate(
    spam_binary = case_when(
      !!sym(target_col) %in% c("y", "yes", "spam", "1") ~ 1,
      !!sym(target_col) %in% c("n", "no", "nonspam", "0") ~ 0,
      TRUE ~ as.numeric(!!sym(target_col))
    )
  )

# --- Step 4: Fit the Linear Probability Model ---
lpm <- lm(spam_binary ~ char_dollar + credit + money + re, data = spam)

summary(lpm)


Rows: 4601 Columns: 58
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (58): make, address, all, 3d, our, over, remove, internet, order, mail, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Unique value counts per column:"
                   is_spam                      table 
                         2                         38 
                        3d                      parts 
                        43                         53 
                      font                        857 
                        99                        106 
                conference                         cs 
                       106                        108 
                       415                    receive 
                       110                        113 
                 addresses                     direct 
                       118                        125 
                    telnet                     report 
                       128                        133 
                  original                       over 
                       136                        141 
                      make                      money 
                       142 


Call:
lm(formula = spam_binary ~ char_dollar + credit + money + re, 
    data = spam)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.8494 -0.3346 -0.3035  0.5141  0.9626 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.334591   0.007322  45.696   <2e-16 ***
char_dollar  0.585510   0.026750  21.889   <2e-16 ***
credit       0.157520   0.012853  12.255   <2e-16 ***
money        0.187942   0.014874  12.635   <2e-16 ***
re          -0.053554   0.006475  -8.271   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.443 on 4596 degrees of freedom
Multiple R-squared:  0.1789,	Adjusted R-squared:  0.1782 
F-statistic: 250.3 on 4 and 4596 DF,  p-value: < 2.2e-16


Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [8]:
# Your code here!
# 1. Fit the Linear Probability Model (LPM)
lpm <- lm(spam_binary ~ char_dollar + credit + money + re, data = spam)

# 2. Predict the in-sample probability for each email
pred_probs <- predict(lpm)

# 3. Find smallest predicted probability
min_prob <- min(pred_probs)
min_prob

# 4. Find largest predicted probability
max_prob <- max(pred_probs)
max_prob

# Optional: look at the first few predicted probabilities
head(pred_probs)



[1] -0.8125404

[1] 3.849409

1         2         3         4         5         6 
0.3345915 0.5207985 0.5007950 0.3345915 0.3345915 0.3345915

## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [9]:
# Your code here!
# 1. Convert probability to odds
prob_to_odds <- function(p) {
  p / (1 - p)
}

# 2. Convert odds to probability
odds_to_prob <- function(odds) {
  odds / (1 + odds)
}

# --- Testing the functions ---

# Test: 2:1 odds should return 2/3 probability
odds_to_prob(2)

# Test: Probability 2/3 should return 2:1 odds
prob_to_odds(2/3)

# --- Doubling your odds example ---
# Your original winning probability is 0.60
p <- 0.60

# Convert to odds
original_odds <- prob_to_odds(p)

# Double the odds
new_odds <- 2 * original_odds

# Convert back to probability
new_probability <- odds_to_prob(new_odds)
new_probability



[1] 0.6666667

[1] 2

[1] 0.75

## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [10]:
model = glm(is_spam ~ 1 + char_dollar + credit + money + re, family='binomial', data=spam)

summary(model)

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”



Call:
glm(formula = is_spam ~ 1 + char_dollar + credit + money + re, 
    family = "binomial", data = spam)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.06656    0.04321 -24.680  < 2e-16 ***
char_dollar 11.81757    0.60451  19.549  < 2e-16 ***
credit       2.31190    0.34298   6.741 1.58e-11 ***
money        1.99328    0.24848   8.022 1.04e-15 ***
re          -0.77550    0.09936  -7.805 5.94e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6170.2  on 4600  degrees of freedom
Residual deviance: 4427.8  on 4596  degrees of freedom
AIC: 4437.8

Number of Fisher Scoring iterations: 7


Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [11]:
library(tidyverse)

# --- Logistic Regression Model ---
model <- glm(is_spam ~ char_dollar + credit + money + re,
             family = "binomial",
             data = spam)

summary(model)

# --- Extract coefficients ---
b <- coef(model)
b

intercept <- b[1]
money_coef <- b["money"]

cat("Intercept (log-odds):", intercept, "\n")
cat("Money coefficient (log-odds):", money_coef, "\n")

# --- Convert to odds scale ---
intercept_odds <- exp(intercept)
money_odds <- exp(money_coef)

cat("Intercept (odds):", intercept_odds, "\n")
cat("Money coefficient (odds multiplier):", money_odds, "\n")

# --- Helper functions for probability/odds ---
prob_to_odds <- function(p) p / (1 - p)
odds_to_prob <- function(odds) odds / (1 + odds)

# --- Convert to probability scale ---
intercept_prob <- odds_to_prob(intercept_odds)

cat("Intercept (probability):", intercept_prob, "\n")

# Divide-by-4 trick for slope interpretation on probability scale
money_prob_change <- money_coef / 4

cat("Approximate change in probability for 1-unit increase in 'money':",
    money_prob_change, "\n")


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”



Call:
glm(formula = is_spam ~ char_dollar + credit + money + re, family = "binomial", 
    data = spam)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.06656    0.04321 -24.680  < 2e-16 ***
char_dollar 11.81757    0.60451  19.549  < 2e-16 ***
credit       2.31190    0.34298   6.741 1.58e-11 ***
money        1.99328    0.24848   8.022 1.04e-15 ***
re          -0.77550    0.09936  -7.805 5.94e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6170.2  on 4600  degrees of freedom
Residual deviance: 4427.8  on 4596  degrees of freedom
AIC: 4437.8

Number of Fisher Scoring iterations: 7


(Intercept) char_dollar      credit       money          re 
 -1.0665628  11.8175673   2.3118984   1.9932803  -0.7755045

Intercept (log-odds): -1.066563 
Money coefficient (log-odds): 1.99328 
Intercept (odds): 0.3441895 
Money coefficient (odds multiplier): 7.33957 
Intercept (probability): 0.2560573 
Approximate change in probability for 1-unit increase in 'money': 0.4983201 
